<a href="https://colab.research.google.com/github/jasreman8/LLMs-For-RAGs/blob/main/RAG_Based_Q%26A_System_for_PowerBI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Business Context

## Implementing a RAG System for Power BI Usage

**Problem Scenario:**

In the current data-driven landscape, organizations increasingly rely on powerful analytics tools like Power BI to derive insights and make informed decisions. However, many analysts struggle with the complexity and breadth of Power BI’s official documentation. The extensive resources often lead to confusion, causing users to misinterpret features or overlook essential functionalities. This challenge can result in inefficient data analysis, wasted time, and missed opportunities for actionable insights. Consequently, analysts may not fully leverage the capabilities of Power BI, stifling potential business growth and impact.

# Objective

To address these challenges, we propose implementing a **Retrieval-Augmented Generation (RAG) system** specifically designed for Power BI. This system will enable analysts to formulate questions using natural language and retrieve concise, relevant answers directly from the official documentation. By facilitating better access to critical information, we aim to enhance the operational efficiency of analysts and empower them to utilize Power BI to its fullest potential.

The RAG application will simplify interactions with Power BI documentation, allowing users to inquire about specific features, functions, or best practices and receive clear explanations in real-time. By improving understanding and accessibility to the tool, analysts will be able to make quicker, data-driven decisions that lead to a significant business impact.

# Installing and Importing the Necessary Libraries

In this section, we need to install and import libraries required to run the notebook:

- The `openai` package provides the official OpenAI API client for accessing models like GPT-4, Whisper, DALL·E, including its embedding models.

- The `tiktoken`	library provides access to OpenAI's tokenizer models, crucial for chunking and token counting.

- The `pypdf` library parses and extracts text from PDF files — useful for document ingestion.

- LangChain is a GenAI framework to build applications with LLMs using chains and agents.
  - `langchain` is the core library that provides access to various LangChain abstractions
  - `langchain-community` provides access to 3rd-party integrations (e.g., different vector stores, tools)
  - `langchain-chroma` provides specific integration to use ChromaDB as the vector store backend in LangChain
  - `langchain-openai` module provides a plug-in interface for LangChain to call OpenAI's LLMs using standardized interface

- `chromadb` library provides access to ChromaDB vector database, which is a fast, vector database optimized for retrieval in RAG systems

# Installing the required libraries
!pip install -q openai \
tiktoken \
pypdf \
langchain \
langchain-community \
langchain-chroma \
langchain-openai \
langchain_text_splitters \
chromadb==0.6.3 \
posthog==2.4.2


**Importing the Libraries**


In [ ]:
# Importing the standard Libraries
import time                           # For measuring execution time or adding delays
from datetime import datetime         # For handling timestamps and datetime operations

# ChromaDB Vector Database
import chromadb  # Chroma: a local-first vector database for storing and querying document embeddings

# OpenAI SDK
from openai import OpenAI
# Official OpenAI Python SDK (v1.x) for interacting with models like GPT-4

# LangChain Utilities
# RecursiveCharacterTextSplitter intelligently breaks long text into smaller chunks with some overlap, preserving context.
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Loads all PDF files from a directory and extracts text from each.
from langchain_community.document_loaders import PyPDFDirectoryLoader

# Base class representing a document in LangChain; useful for downstream chaining and processing.
from langchain_core.documents import Document

# Embeddings and Vector Store
# Generates vector embeddings using OpenAI’s embedding models (e.g., `text-embedding-3-small`)
from langchain_openai import OpenAIEmbeddings

# Integration for using Chroma as the vector store within LangChain’s ecosystem
from langchain_chroma import Chroma

from google.colab import userdata
import os, shutil

import warnings
warnings.filterwarnings('ignore')

## Setup the API Key
#### Setup the OpenAI API key and initialize the client with the required model.

In [ ]:
# Set up the OpenAI API Key
openai_api_key = userdata.get('my_api_key')

client = OpenAI(
    api_key=openai_api_key,
)

model_name = 'gpt-4o-mini'

embedding_model = OpenAIEmbeddings(
    api_key=openai_api_key,
    model='text-embedding-3-small'
)

## Creating Vector Database

In [ ]:
# Unzip the dataset containing the policy document
!unzip PowerBI.zip

Archive:  PowerBI.zip
  inflating: Introducing_Power_BI.pdf  


## Load PDF Documents and perform chunking

In this step, I will:

- Load PDF documents from folder where pdf are saved using PyPDFDirectoryLoader.

- Split documents into chunks using RecursiveCharacterTextSplitter with the specified tokenizer, chunk size, and overlap.

- Store chunks within LangChain’s Document class.

- Inspect contents of the first page by accessing its .page_content attribute.

- Define a ChromaDB collection name to store the chunks for later retrieval.

In [ ]:
# Set the directory where PDF files to be stored
pdf_folder_location = "powerbi_docs"

# Initialize a PDF loader to load all PDF documents in the directory
pdf_loader = PyPDFDirectoryLoader(pdf_folder_location)

In [ ]:
# Define the temporary folder name
temp_folder = "powerbi_docs"

# Create the temporary folder if it doesn't exist
if not os.path.exists(temp_folder):
    os.makedirs(temp_folder)

# Define the source and destination paths for the PDF file
source_path = "Introducing_Power_BI.pdf"
destination_path = os.path.join(temp_folder, "Introducing_Power_BI.pdf")

# Move the PDF file into the temporary folder
shutil.move(source_path, destination_path)

print(f"Moved '{source_path}' to '{destination_path}'")

Moved 'Introducing_Power_BI.pdf' to 'powerbi_docs/Introducing_Power_BI.pdf'


In [ ]:
pdf_loader

In [ ]:
# Splitting documents into chunks using RecursiveCharacterTextSplitter with the specified tokenizer, chunk size, and overlap
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',  # OpenAI tokenizer for accurate token count
    chunk_size=512,               # Limit each chunk to 512 tokens
    chunk_overlap=16              # Keep 16-token overlap between consecutive chunks
)

In [ ]:
# Load the PDF documents and split them into chunks using the text splitter
powerbi_chunks = pdf_loader.load_and_split(text_splitter)

#Investigating the type of power_bi chunks
type(powerbi_chunks)

list

In [ ]:
# Investigating the length of power_bi chunks
len(powerbi_chunks)

407

In [ ]:
# Investigating the chunks are stored within LangChain's Document class
powerbi_chunks[0]

Document(metadata={'producer': 'Adobe Acrobat Pro 10.1.16', 'creator': 'Adobe Acrobat Pro 10.1.16', 'creationdate': '2016-06-13T10:18:21-04:00', 'author': 'Joan', 'moddate': '2016-06-13T21:13:38-04:00', 'title': '', 'source': 'powerbi_docs/Introducing_Power_BI.pdf', 'total_pages': 407, 'page': 0, 'page_label': '1'}, page_content='Introducing\nMicrosoft \nPower BI\nAlberto Ferrari and Marco Russo')

In [ ]:
# The individual text chunks can be accessed via the `.page_content` attribute of the `Document` class
# Inspecting the contents of the first page by accessing its .page_content attribute
powerbi_chunks[0].page_content

'Introducing\nMicrosoft \nPower BI\nAlberto Ferrari and Marco Russo'

In [ ]:
# The metadata corresponding to the first chunk can be accessed via the .metadata attribute
powerbi_chunks[0].metadata

{'producer': 'Adobe Acrobat Pro 10.1.16',
 'creator': 'Adobe Acrobat Pro 10.1.16',
 'creationdate': '2016-06-13T10:18:21-04:00',
 'author': 'Joan',
 'moddate': '2016-06-13T21:13:38-04:00',
 'title': '',
 'source': 'powerbi_docs/Introducing_Power_BI.pdf',
 'total_pages': 407,
 'page': 0,
 'page_label': '1'}

In [ ]:
# Let's define the ChromaDB collection name to store the chunks
powerbi_collection = 'powerbi-health-pulse'

### Initialize the OpenAI embedding model with the API key, endpoint, and embedding model name.
In this step, I will:

- Instantiate the OpenAI embedding model with my API key, endpoint, and embedding model name.

- Initialize a persistent Chroma client for managing embeddings.

- Ping the database client using the heartbeat method to confirm the connection is alive.

- Verify the database is empty before adding new embeddings.

- Create a Chroma vector store to store and retrieve document embeddings.

- Confirm the collection creation and that the database has been populated.

- Batch process 500 chunks at a time when sending to the API, and pause execution for 30 seconds after each batch to avoid rate limits.

In [ ]:
# Instantiating the OpenAI embedding model
embedding_model = OpenAIEmbeddings(
    api_key = openai_api_key, # OpenAI API key
    model = 'text-embedding-3-small' # OpenAI's lightweight and cost-effective embedding model
)

In [ ]:
# Initialize a persistent Chroma client
chromadb_client = chromadb.PersistentClient(
    path="./powerbi_db", # Local directory where vector database will be stored
 )

In [ ]:
# Pinging the database client to check if the connection is alive
# the heartbeat method returns the current time in nanoseconds and is generally used to check if the server is alive
chromadb_client.heartbeat()

1768435323215648187

In [ ]:
# Confirm database is empty
chromadb_client.list_collections()

['powerbi-health-pulse']

In [ ]:
chromadb_client.count_collections()

1

In [ ]:
# Instantiate a Chroma vector store to store and retrieve document embeddings
vectorstore = Chroma(
    collection_name = powerbi_collection,               # Name of the Chroma collection to group related embeddings
    collection_metadata = {"hnsw:space": "cosine"},     # Use cosine similarity for nearest-neighbor search (HNSW index)
    embedding_function = embedding_model,               # Embedding model used to convert text into vectors
    client = chromadb_client,                           # Persistent Chroma client initialized earlier
    persist_directory = "./powerbi_db"                  # Directory where Chroma will persist its data
)

In [ ]:
# confirm collection creation
chromadb_client.list_collections()

['powerbi-health-pulse']

In [ ]:
# Confirm database has been populated with the collection
chromadb_client.count_collections()

1

In [ ]:
# Batch 500 chunks to send to the API at a time, pausing execution for 30 seconds afterward
i = 0 # Initialize the starting index for the chunks

while i < len(powerbi_chunks): # Iterate while the index is less than the total number of chunks
    vectorstore.add_documents( # Add documents to the vector store in batches of 500
        documents=powerbi_chunks[i:i+500], # Get the current batch of 500 chunks
        ids=["text_" + str(i) for i in range(i, i+500)] # Assign unique IDs to each chunk in the batch
    )

    i += 500 # Move to the next batch by incremening the index by 500
    time.sleep(30) # Pause for 30 seconds to avoid rate limiting issues with the vector store

# CRUD Operations in ChromaDB




## **READ**

Once the database is created, the stored entries can be retrieved by initializing a new Chroma instance (denoted as **vectorstore_persisted** to distinguish between creation and read operations) and directing it to the persistent storage directory containing the document embeddings.

In this step, I will:

Initialize a new Chroma instance (e.g., vectorstore_persisted) and point it to the persistent storage directory where embeddings are stored.

In [ ]:
# Initializing a new Chroma instance denoted as vectorstore_persisted
vectorstore_persisted = Chroma(
    collection_name = powerbi_collection,
    collection_metadata = {"hnsw:space": "cosine"},
    embedding_function = embedding_model,
    client = chromadb_client,
    persist_directory = "./powerbi_db"
)

There are two valuable types of READ operations in vector databases:

1. **Inspecting individual records**
2. **Retrieving relevant records based on a user query**

In this step, I will:

 - Define the Chroma collection to work with.

 - Count the number of records present in the collection.

Inspect the first 2 records using the .peek() method to confirm that embeddings have been stored correctly.

**Inspecting individual records**

In [ ]:
# Define the chroma collection
collection = chromadb_client.get_collection(powerbi_collection)

# Count the number of records in the collection
collection.count()

407

In [ ]:
# Inspect the first 2 records using the .peek() method
collection.peek(2)

{'ids': ['text_0', 'text_1'],
 'embeddings': array([[-0.0141233 , -0.03104495,  0.0387464 , ..., -0.00077508,
          0.00032513,  0.01716083],
        [ 0.0097238 ,  0.00484162,  0.02150327, ..., -0.019948  ,
         -0.01871731,  0.01614773]]),
 'documents': ['Introducing\nMicrosoft \nPower BI\nAlberto Ferrari and Marco Russo',
  'PUBLISHED BY \nMicrosoft Press \nA division of Microsoft Corporation \nOne Microsoft Way \nRedmond, Washington 98052-6399 \nCopyright © 2016 by Microsoft Corporation \nAll rights reserved. No part of the contents of \nthis book may be reproduced or transmitted in \nany form or by any means without the written \npermission of the publisher. \nISBN: 978-1-5093-0228-4 \nMicrosoft Press books are available through \nbooksellers and distributors worldwide. If you \nneed support related to this book, email \nMicrosoft Press Support at \nmspinput@microsoft.com. Please tell us what \nyou think of this book at http://aka.ms/tellpress. \nThis book is provided “as-

In [ ]:
# Inspect a specific record in the collection
collection.get(
    ids = ['text_234']
)

{'ids': ['text_234'],
 'embeddings': None,
 'documents': ['215 CH A P T E R  5  |  Getting data from services and  \n content packs \n \n \nFigure 5-20: The Create Content Pack dialog box \nreq\nuires you to select objects to publish in a new \ncontent pack. \nWhen you click the Publish button, the content \npack is published and displayed in the list of the \ncontent packs that you can obtain by selecting \nthe View Content Pack item in the Settings menu \n(refer to Figure 5-19).'],
 'uris': None,
 'data': None,
 'metadatas': [{'author': 'Joan',
   'creationdate': '2016-06-13T10:18:21-04:00',
   'creator': 'Adobe Acrobat Pro 10.1.16',
   'moddate': '2016-06-13T21:13:38-04:00',
   'page': 234,
   'page_label': '235',
   'producer': 'Adobe Acrobat Pro 10.1.16',
   'source': 'powerbi_docs/Introducing_Power_BI.pdf',
   'title': '',
   'total_pages': 407}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

## Observations
The RAG chatbot was made to be scalable and resumable.
 - The PowerBI documents are ingested and embedded.
 - The Chroma loads the existing vectors instantly.
 - During retrieval, the retriever queries vectorstore_persisted for the top-k most semantically similar chunks to a user's question.
 - While inspecting individual records, it was observed that there were 407 records.
 - First two records were inspected using peek method, and a specific random record was inspected too.

**Retrieving relevant records based on a user query**

The primary function of the vector database is to retrieve relevant records based on user queries and to facilitate this process, we implement a retriever that utilizes the query embeddings to query the database.

Write code that uses HNSW algorithm to calculate the nearest neighbors for the user query and returns the corresponding documents from the database.

In [ ]:
# Create a retriever interface from the vector store
retriever = vectorstore_persisted.as_retriever(
    search_type = 'similarity',             # Use the default method based on semantic similarity
    search_kwargs = {'k': 5}                # Retrieve top 5 most similar chunks
)

In [ ]:
# Define a sample user query
user_query = "How can I create a calculated measure in Power BI using DAX?"

Write code to  performs the similarity search based on the user query by using the `.invoke()` method.

In [ ]:
# Perform similarity search to return the top 5 document chunks based on the sample user query
# Replace 'user_query' with the actual query you want to search for
retriever.invoke(user_query)

[Document(id='text_227', metadata={'author': 'Joan', 'creationdate': '2016-06-13T10:18:21-04:00', 'creator': 'Adobe Acrobat Pro 10.1.16', 'moddate': '2016-06-13T21:13:38-04:00', 'page': 227, 'page_label': '228', 'producer': 'Adobe Acrobat Pro 10.1.16', 'source': 'powerbi_docs/Introducing_Power_BI.pdf', 'title': '', 'total_pages': 407}, page_content='208 CH A P T E R  5  |  Getting data from services and  \n content packs \n \nBecause David used Power Pivot for Excel in the \npast, he already knows how to write the measure \nhe needs. So, on the ribbon, on the Home tab, \nhe clicks New Measure and inserts the following \nDAX measure in the formula bar: \nNew Users Growth = \nIF ( \n    HASONEVALUE ( Website[Year] ), \n    DIVIDE ( \n        SUM ( Website[New Users] ), \n        CALCULATE ( \n            SUM ( Website[New Users] ), \n            Website[Year] = VALUES ( Website[Year] \n) - 1 \n        ) \n    ) \n) \nThen, he displays this measure in a separate \nvisualization, under the

## Observations
At the retrieval stage of the RAG pipeline, we can start testing how well the PowerBI documentation embeddings work.
 - The retriever embeds this query.
 - It searches the Chroma collection for top 5 semantically similar chunks (based on Cosine similarity) related to creating DAX measures.

# RAG Q&A System for PowerBI Documentation

A typical RAG implementation consists of the following stages:
* Indexing Stage
* Retrieval Stage
* Generation Stage

| Stage          | Key Activities                                        | Role in RAG                              |
| -------------- | ----------------------------------------------------- | ---------------------------------------- |
| **Indexing**   | Chunking · Embedding · Storing                        | Prepares data for efficient retrieval    |
| **Retrieval**  | Query embedding · Similarity search   | Consolidates relevant context            |
| **Generation** | Prompt construction · LLM generation | Produces final response grounded in data |


Let's now put together the RAG pipeline using these stages.



## Retrieval Stage

**Retrieving Relevant Documents**

Write code that performs the Retrieval stage in the RAG pipeline.

 define a sample user query to test the RAG pipeline

In [ ]:
user_query = 'What is Direct Lake mode in Power BI and how is it different from DirectQuery?'

Retrieve the relevant chunks from the documents based on the `user_query`.


In [ ]:
relevant_document_chunks = retriever.invoke(user_query)
len(relevant_document_chunks)

5

In [ ]:
# Inspecting the first document
for document in relevant_document_chunks:
    print(document.page_content.replace("\t", " "))
    break

138 C H A P T E R  4  |  Using Power BI Desktop 
 
At first glance, it looks like DirectQuery is the 
most convenient method for loading data, but 
this is not totally true. If the data is updated 
frequently, it is very likely that one minute you 
will see a report with a set of figures, but when 
you open it again a few minutes later, the 
numbers might no longer be the same. This is 
frustrating if you are analyzing information over 
the span of an entire year (which is what David is 
doing). Numbers that change too frequently can 
become disturbing. Also, although real-time data 
might sometimes be useful, it comes at the cost 
of query speed; DirectQuery by its very nature is 
much slower than working with data that is 
resident on your device and directly accessible 
by Power BI Desktop. 
As a final note, keep in mind that DirectQuery 
works fine when you use Power BI Desktop on 
your laptop, but when you publish the model to 
Power BI, the cloud service needs a way to 
communica

## Generation Stage

This section will perform the **Generation** stage of the RAG pipeline.

We will pass the relevant context chunks to the LLM, along with the system message and user message via a prompt template.

These are then passed to the LLM to compose an appropriate response to the user's query.


### Prompt Template

Define the system message for the RAG chatbot with the appopriate role, context and the relevant instructions.

In [ ]:
qna_system_message = """
You are a Power BI expert assisting a business division. Answer user questions about Power BI features and practices.
STRICT RULES:
1) Use ONLY the information provided between <Context>...</Context>. Do not use outside knowledge or speculate.
2) If the answer cannot be found, reply exactly: I don't know
3) Do not reference or mention the existence of any context, documents, sources, or citations.
4) Be concise and practical. Prefer step-by-step instructions or bullet points when helpful.
5) If the question is ambiguous or missing key details, ask ONE clarifying question first, then wait for the user’s reply.

FORMATTING:
- Plain text or Markdown only.
- If you include code, use proper fenced blocks:
  - DAX: ```DAX
  - Power Query (M): ```PowerQuery
- Use numbered steps for procedures; short bullets for options/constraints.
- Avoid filler, apologies, or meta commentary.

POWER BI SCOPE HINTS (if present in the context):
- Calculations: measures vs calculated columns; common DAX (CALCULATE, FILTER, TOTALYTD, ALL, RELATED).
- Modeling: relationships (single/bidirectional), star schema, composite models.
- Performance: Import vs DirectQuery vs Direct Lake; Performance Analyzer basics.
- Security: row-level security (RLS), dynamic roles.
- Refresh: gateways, scheduled refresh, incremental refresh.

Answer policy summary: answer strictly from <Context>; otherwise say I don't know; keep it short and useful.
""".strip()

Write the user message prompt template that provides the relevant chunks and the user query within the `context` and `question` placeholders respectively.

In [ ]:
qna_user_message_template = """
<Context>
Here are some documents that are relevant to the question mentioned below:
{context}
</Context>

<Question>
{question}
</Question>
"""

### Generating the Response
In this step, you need to:   

Prompt construction, LLM API call with error handling, and response parsing

In [ ]:
user_query = 'How do I implement row-level security for users in a Power BI dataset?'

relevant_document_chunks = retriever.invoke(user_query)
context_list = [document.page_content for document in relevant_document_chunks]
context_for_query = "\n---\n".join(context_list)

prompt = [
    {'role':'developer', 'content':qna_system_message},
    {'role':'developer', 'content':qna_user_message_template.format(context=context_for_query, question=user_query)}
]

try:
  response = client.chat.completions.create(
      model = model_name,
      messages = prompt,
      temperature = 0
  )
  prediction = response.choices[0].message.content.strip()
except Exception as error:
  prediction = f'Sorry, I encountered the following error: \n {error}'

print(prediction)

1. **Open Power BI Desktop** and load your dataset.
2. **Go to the Modeling tab** in the ribbon.
3. **Select Manage Roles** to create security roles.
4. **Click on Create** to define a new role.
5. **Set DAX filters** for each table to specify which rows users in this role can see. For example, to restrict access to sales data for China, you might use a DAX expression like:
   ```DAX
   [Country] = "China"
   ```
6. **Add members** to the role by selecting the role and clicking on "Add members."
7. **Publish the dataset** to the Power BI service.
8. **In the Power BI service**, go to the dataset settings and select the Security option to manage roles and assign users.

Note: Ensure that row-level security is applied only to datasets created in Power BI Desktop, as it cannot be applied to datasets created with Power Pivot for Excel.


## Observation
This is the core "generation" step of RAG pipeline where the retrieved document chunks (from the retriever) are sent to the LLM to produce the final natural-language answer to the user's question.

- A system prompt (qna_system_message) is defined to constrain the model's behavior.
- The retrieved document chunks and user questions are formatted into a structured input.
- The OpenAI chat model is called to generate an answer.
- The final response is printed.

It is the generation part of the RAG flow:

Retrieval --> Context formatting --> Generation (LLM call)

This block of code feeds the retrieved PowerBI documentation into the LLM, wraps it with clear boundaries and instructions, and returns a clean, grounded answer.

# Putting it all together - PowerBI RAG Q&A Chatbot

We'll now put together the relevant codes for the RAG pipeline into a file named `rag-chat.py` to create a basic command-line chat interface which can run via  the terminal.

This naive RAG implementation illustrates how document Q&A could be automated for any domain.

Write code that use the `%%writefile` magic command specific to Google Colab, which allows the content of a cell to be written directly into a file on the virtual machine's disk.

This allows for the creation of scripts, configuration files, or data files within the Colab environment. These files are available during the Colab runtime and are deleted when the runtime is stopped or deleted.

The `!python` shell command can be used to execute a Python script (.py files) or commands within the Colab environment.

In [ ]:
%%writefile rag_chat.py
import os, sys, logging, chromadb
from typing import List
from openai import OpenAI
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

logging.getLogger("chromadb").setLevel(logging.CRITICAL)

MODEL_NAME = "gpt-4o-mini"
POWERBI_COLLECTION = "powerbi-health-pulse"
PERSIST_DIR = "./powerbi_db"
TOP_K = 5
MAX_CHARS_PER_CHUNK = 1600  # light guardrail against token bloat

# -------- API key resolution --------
def resolve_api_key() -> str:
    if len(sys.argv) > 1 and sys.argv[1].strip():
        return sys.argv[1].strip()
    env_key = os.getenv("OPENAI_API_KEY")
    if env_key:
        return env_key
    print("Error: Provide OpenAI API key as argv[1] or set OPENAI_API_KEY env var.")
    sys.exit(1)

OPENAI_API_KEY = resolve_api_key()

# -------- Clients --------
client = OpenAI(
    api_key=OPENAI_API_KEY,
)

embedding_model = OpenAIEmbeddings(
    api_key=OPENAI_API_KEY,
    model="text-embedding-3-small",
)

chromadb_client = chromadb.PersistentClient(path=PERSIST_DIR)

vectorstore_persisted = Chroma(
    collection_name=POWERBI_COLLECTION,
    collection_metadata={"hnsw:space": "cosine"},
    embedding_function=embedding_model,
    client=chromadb_client,
    persist_directory=PERSIST_DIR,
)

retriever = vectorstore_persisted.as_retriever(
    search_type="similarity",
    search_kwargs={"k": TOP_K},
)

# -------- Prompting --------
qna_system_message = """
You are a Power BI expert assisting a business division. Answer user questions about Power BI features and practices.
STRICT RULES:
1) Use ONLY the information provided between <Context>...</Context>. Do not use outside knowledge or speculate.
2) If the answer cannot be found, reply exactly: I don't know
3) Do not reference or mention the existence of any context, documents, sources, or citations.
4) Be concise and practical. Prefer step-by-step instructions or bullet points when helpful.
5) If the question is ambiguous or missing key details, ask ONE clarifying question first, then wait for the user’s reply.

FORMATTING:
- Plain text or Markdown only.
- If you include code, use proper fenced blocks:
  - DAX: ```DAX
  - Power Query (M): ```PowerQuery
- Use numbered steps for procedures; short bullets for options/constraints.
- Avoid filler, apologies, or meta commentary.

POWER BI SCOPE HINTS (if present in the context):
- Calculations: measures vs calculated columns; common DAX (CALCULATE, FILTER, TOTALYTD, ALL, RELATED).
- Modeling: relationships (single/bidirectional), star schema, composite models.
- Performance: Import vs DirectQuery vs Direct Lake; Performance Analyzer basics.
- Security: row-level security (RLS), dynamic roles.
- Refresh: gateways, scheduled refresh, incremental refresh.

Answer policy summary: answer strictly from <Context>; otherwise say I don't know; keep it short and useful.
""".strip()

QNA_USER_MESSAGE_TEMPLATE = """
<Context>
Here are some documents that are relevant to the question mentioned below:
{context}
</Context>

<Question>
{question}
</Question>
""".strip()

def _truncate(s: str, max_chars: int) -> str:
    return s if len(s) <= max_chars else s[:max_chars] + "..."

def build_context(chunks: List[str]) -> str:
    # Trim each chunk lightly and join with a separator
    safe_chunks = [_truncate(c, MAX_CHARS_PER_CHUNK) for c in chunks]
    return "\n---\n".join(safe_chunks)

def respond_to_query(user_query: str) -> str:
    try:
        # Prefer widely supported API
        docs = retriever.invoke(user_query) # Changed from .get_relevant_documents
    except Exception as e:
        return f"Sorry, I encountered a retrieval error: {e}"

    context_list = [d.page_content for d in docs] if docs else []
    context_for_query = build_context(context_list)

    messages = [
        {"role": "system", "content": qna_system_message},
        {"role": "user", "content": QNA_USER_MESSAGE_TEMPLATE.format(
            context=context_for_query, question=user_query)},
    ]

    try:
        resp = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
            temperature=0,
        )
        answer = resp.choices[0].message.content.strip()
        if not answer:
            return "I don't know"
        return answer
    except Exception as error:
        return f"Sorry, I encountered the following error:\n{error}"

def get_sources(user_query: str, k: int = TOP_K):
    try:
        docs = retriever.invoke(user_query) # Changed from .get_relevant_documents
    except Exception:
        return []
    items = []
    for d in docs[:k]:
        meta = getattr(d, "metadata", {}) or {}
        items.append({
            "title": meta.get("title") or meta.get("url") or "Source",
            "url": meta.get("url"),
            "snippet": d.page_content[:500] + ("..." if len(d.page_content) > 500 else "")
        })
    return items

def main():
    """
    Runs the main interactive loop for the Q&A system.
    """
    print("PowerBI RAG Q&A. Type 'q' to quit.\n")
    try:
        while True:
            user_query = input("User: ").strip()
            if user_query.lower() in {"q", "quit", "exit"}:
                print("Goodbye!")
                break
            answer = respond_to_query(user_query)
            print(f"Assistant: {answer}\n")
    except KeyboardInterrupt:
        print("\nGoodbye!")

if __name__ == "__main__":
    main()

Overwriting rag_chat.py


Run the script using the `!python` shell command.

Fomulate 5 queries on the PowerBI Documentation that will then be used to validate the the Q&A RAG Chatbot and provide the output responses.

**SAMPLE QUESTIONS**

- How do I share dashboards securely with other users?
- How do I handle missing or null data in Power Query?
- How do I share dashboards securely with colleagues in Power BI Service?
- What is a Power BI Gateway and when do I need one?
- When should I use Import mode vs. DirectQuery mode?
- What is a Power BI Gateway and when do I need one?
- How can I use REST APIs to manage Power BI workspaces?
- What are the key differences between Power BI Desktop and Power BI Service?

In [ ]:
# Run the script and pass the API key as a command-line argument
!python rag_chat.py "$openai_api_key"

PowerBI RAG Q&A. Type 'q' to quit.

User: How do I share dashboards securely with other users?
Assistant: To share dashboards securely with other users, follow these steps:

1. **Identify User Type**:
   - **Internal Users**: Users within your organization.
   - **External Users**: Users outside your organization.

2. **Sharing with Internal Users**:
   - Invite them by email or send them the dashboard URL.
   - Ensure they are authorized to access the dashboard. If not, they can request permission.

3. **Sharing with External Users**:
   - Invite them by email only.
   - The external user must sign in to Power BI using the same email account used in the invitation.
   - If they have not used Power BI before, they can create a free account upon signing in.

4. **Using Dashboard Links**:
   - You can copy the dashboard URL from the Access tab and send it via your own email if you prefer not to use the Power BI service for notifications.

5. **Permissions**:
   - By default, users will h